In [33]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [34]:
cols = ['name', 'date', 'address', 'infoLink']
catagoryIndexDict = {'2': 'Learning', '4': 'Performance', '11': 'Exhibition', '7': 'Mixer', '8': 'Foods', '6': 'Outdoors'}
catagoryValueDict = {'Learning': None, 'Performance': None, 'Exhibition': None, 'Mixer': None, 'Outdoors': None}

In [35]:
def parseEventData(urlBase, catagory):
    catagoryValue = catagoryIndexDict[catagory]
    pageUrl = ''
    maxPage = 30
    
    df_name = pd.DataFrame(columns = [cols[0]])
    df_date = pd.DataFrame(columns = [cols[1]])
    df_address = pd.DataFrame(columns = [cols[2]])
    df_info = pd.DataFrame(columns = [cols[3]])
    print('Running catagory:', catagoryValue)
    for page in range(2, maxPage):
        res = requests.get(urlBase + catagory + pageUrl)         
        soup = soup = BeautifulSoup(res.text, "lxml")

        rawDateData = soup.select('.timezoneSuffix')
        rawEventNameData = soup.select('h2 a')
        rawInfoHref = soup.select('.btn-point')

        for rawDate in rawDateData:
            date = rawDate.get_text()
            df_date = df_date.append({'date': date}, ignore_index=True)

        for event in rawEventNameData:
            eventName = event.get_text()
            df_name = df_name.append({'name': eventName}, ignore_index=True)

        for a in soup.find_all('a', href=True, attrs={'class': 'btn-point btn-small pull-right'}):
            eventInfoLink = a['href']
            df_info = df_info.append({'infoLink': eventInfoLink}, ignore_index=True)

            infoRes = requests.get(eventInfoLink)
            infoSoup = BeautifulSoup(infoRes.text, "lxml")
            for span in infoSoup.find_all('span', attrs={'class': 'info-desc'}):
                if span.findChildren('i', attrs={'class': 'fa fa-map-marker'}):
                    address = span.get_text()
                    df_address = df_address.append({'address': address}, ignore_index=True)
        pageUrl = '&page=' + str(page)
        print('Running page:', page)
                    
    df = pd.concat([df_name, df_date, df_address, df_info], axis=1)
    df = df.replace(r'^\\s*$', np.nan, regex=True)
    df = df.dropna()
    df = df.to_dict()
    catagoryValueDict[catagoryValue] = df

In [36]:
catagories = ['2', '4', '11', '7', '8' ,'6']

for catagory in catagories:
    urlBased = 'https://kktix.com/events?category_id='
    parseEventData(urlBased, catagory)

Running catagory: Learning
Running page: 2
Running page: 3
Running page: 4
Running page: 5
Running page: 6
Running page: 7
Running page: 8
Running page: 9
Running page: 10
Running page: 11
Running page: 12
Running page: 13
Running page: 14
Running page: 15
Running page: 16
Running page: 17
Running page: 18
Running page: 19
Running page: 20
Running page: 21
Running page: 22
Running page: 23
Running page: 24
Running page: 25
Running page: 26
Running page: 27
Running page: 28
Running page: 29
Running page: 30
Running page: 31
Running page: 32
Running page: 33
Running page: 34
Running page: 35
Running page: 36
Running page: 37
Running page: 38
Running page: 39
Running page: 40
Running page: 41
Running page: 42
Running page: 43
Running page: 44
Running page: 45
Running page: 46
Running page: 47
Running page: 48
Running page: 49
Running page: 50
Running page: 51
Running page: 52
Running page: 53
Running page: 54
Running page: 55
Running page: 56
Running page: 57
Running page: 58
Running page

Running page: 87
Running page: 88
Running page: 89
Running page: 90
Running page: 91
Running page: 92
Running page: 93
Running page: 94
Running page: 95
Running page: 96
Running page: 97
Running page: 98
Running page: 99
Running catagory: Outdoors
Running page: 2
Running page: 3
Running page: 4
Running page: 5
Running page: 6
Running page: 7
Running page: 8
Running page: 9
Running page: 10
Running page: 11
Running page: 12
Running page: 13
Running page: 14
Running page: 15
Running page: 16
Running page: 17
Running page: 18
Running page: 19
Running page: 20
Running page: 21
Running page: 22
Running page: 23
Running page: 24
Running page: 25
Running page: 26
Running page: 27
Running page: 28
Running page: 29
Running page: 30
Running page: 31
Running page: 32
Running page: 33
Running page: 34
Running page: 35
Running page: 36
Running page: 37
Running page: 38
Running page: 39
Running page: 40
Running page: 41
Running page: 42
Running page: 43
Running page: 44
Running page: 45
Running page

In [40]:
catagoryValueDict['Learning']

{'name': {0: '吸引人的說話技巧～小蛙老師的自在表達課',
  1: '[ 2019.04.13 開課！]【寫作小學堂】～寫出專屬風格，找回文字悸動~第二期',
  2: '不費力的發聲技巧～小蛙老師的輕鬆說話課',
  3: '[2019.03.08 開課！] 《學「問」～高難度對話的望聞問切》~第15期',
  4: '電子電路+MCU單晶片韌體設計 — ARM Cortex M微控制器培訓基地【免費諮詢】',
  5: '健美行動小學堂愛分享系列講座-1月愛健康主題：育兒好好玩',
  6: '黃勝堅院長《希望你用不到，但一定要知道的長照 》主題講座',
  7: '國立交通大學107年交通安全宣導',
  8: 'DNA 讀人術',
  9: 'How to Identify Your Career Potential  工作的新可能：你不曾發現的自己',
  10: 'Leadership 領導力 : 以同理心共創團隊成就',
  11: 'LiK: MeetUp 2018 暖暖冬日午茶聚會',
  12: '【富邦講堂】《Pantone的色票地圖─年度色來了！》Leatrice Eiseman',
  13: '【愛情命運大作戰系列】 壞情人的20道陰影',
  14: '夢想清單：終身可用的美好人生圓夢指南',
  15: '「心想事成的秘密 - 美國心靈科技探索」 免費公益講座 ',
  16: '《劉軒-斜槓青年進行事》台中免費達人講座',
  17: 'How to Make Love Last  愛情長效藥',
  18: '冬戀的滋味 義式獵人燉雞套餐',
  19: '3rd CEREC Ultimate Experience',
  20: '台灣駭客年會 HITCON Pacific 2018',
  21: '[2018/12/12開課!]《人際回應力-看懂情緒，輕鬆對談》~(第18期)',
  22: 'Innovation 創新力 : 擊潰框架，實踐新可能',
  23: '金融科技與資料科學系列講座-人工智慧簡介與實作',
  24: '《本質思考》_心理學讀書會_選書012',
  25: '高級心臟救命術(ACLS)訓練課程',
  26: '金融科技與資料科學系列講座-以Python實作資料科學',
  27: